In [1]:
# may need to import sys to fix libs pathing for this file
import sys
sys.path.append('../')
from libs.kraken_conn import kraken
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
columns = {0:'datetime',1:'open',2:'high',3:'low',4:'close',5:'volume'}
def make_df(raw_df):
    df = raw_df.loc[:,:5]
    df.rename(columns=columns, inplace=True)
    df = df[["datetime", "close"]]
    
    df["timestamp"] = df["datetime"].apply(lambda x: dt.datetime.fromtimestamp(x/1000))
    df = df.drop(columns="datetime")
    df.set_index("timestamp", inplace=True)
    df.index=pd.to_datetime(df.index)
    return df

In [3]:
# fetchOHLCV will get ~last 30 days of hourly data
# bl = dt.datetime.fromtimestamp((k[0][0])/1000)
def get_historical(pair="BTC/USD"):
    k = kraken.fetchOHLCV(pair, "1h")
    
    df = pd.DataFrame()
    df = df.append(k)
    return make_df(df)

In [4]:
btc_curr = get_historical()
btc_curr.head()

,close
timestamp,
2021-06-01 19:00:00,36180.6
2021-06-01 20:00:00,36280.5
2021-06-01 21:00:00,36306.8
2021-06-01 22:00:00,36488.5
2021-06-01 23:00:00,36690.2


In [5]:
eth_curr = get_historical(pair="ETH/USD")
eth_curr.head()

,close
timestamp,
2021-06-01 19:00:00,2586.66
2021-06-01 20:00:00,2590.44
2021-06-01 21:00:00,2593.15
2021-06-01 22:00:00,2615.78
2021-06-01 23:00:00,2625.73


In [6]:
btc_curr.rename(columns={"close":"btc"}, inplace=True)
eth_curr.rename(columns={"close":"eth"}, inplace=True)

In [7]:
crypto_df = btc_curr.merge(eth_curr, how="inner", on="timestamp")

In [8]:
crypto_df.head()

,btc,eth
timestamp,,
2021-06-01 19:00:00,36180.6,2586.66
2021-06-01 20:00:00,36280.5,2590.44
2021-06-01 21:00:00,36306.8,2593.15
2021-06-01 22:00:00,36488.5,2615.78
2021-06-01 23:00:00,36690.2,2625.73


In [9]:
num_ticks = 60
symbols = ["btc","eth"]
shifted_df = pd.DataFrame()
for symbol in symbols:
    shifted_df[symbol] = crypto_df[symbol].copy()
    for i in range(1,num_ticks):
        name = f"{symbol}-{i}"
        shifted_df[name] = crypto_df[symbol].shift(i)
shifted_df.shape

(720, 120)

In [14]:
curr_df = shifted_df.tail(1)

(660, 120)